In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install wandb

In [ ]:
# 🐝 Secrets
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb")

! wandb login $secret_value_0

Imports

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import os
import cv2
import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()
import shutil
import random
import yaml
import torch


Preliminary folder analysis

In [ ]:
# image_path='/kaggle/input/tensorflow-great-barrier-reef/train_images/video_'
# for i in range(3):
#     filetypes=set()
#     print("for folder video",i,"available file types are: ")
#     for dirname, _, filenames in os.walk(image_path+str(i)):
#         for filename in filenames:
#             ext = os.path.splitext(filename)[-1].lower()
#             filetypes.add(ext)
#     print(filetypes)


# for dirpath, dirname, filenames in os.walk('/kaggle/input/tensorflow-great-barrier-reef'):
#     for file in filenames: 
#         if not file.endswith('.jpg'):
#                               print(file)



In [ ]:
IMG_WIDTH=1280
IMG_HEIGHT=720

In [ ]:
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
IMAGE_DIR = '/kaggle/images' # directory to save images
LABEL_DIR = '/kaggle/labels' # directory to save labels

#creating image and labels directories
if not os.path.exists(IMAGE_DIR):
    !mkdir -p {IMAGE_DIR}
if not os.path.exists(LABEL_DIR):
    !mkdir -p {LABEL_DIR}

In [ ]:
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
# Adding image path and label paths to each image
df['original_path']=f'{ROOT_DIR}/train_images/video_'+df.video_id.astype(str)+'/'+df.video_frame.astype(str)+'.jpg'
#'/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/0.jpg' =>for first data point

# New images and labels path
df['image_path']  = f'{IMAGE_DIR}/'+df.image_id+'.jpg'
df['label_path']  = f'{LABEL_DIR}/'+df.image_id+'.txt'
df['annotations'] = df['annotations'].progress_apply(eval)

display(df.head(20))
print(df.shape[0])

In [ ]:
# COunts number of objectless images
df['num_bbox'] = df['annotations'].progress_apply(lambda x:len(x))
data = (df.num_bbox>0).value_counts(normalize=True)*100
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

In [ ]:
#Removing some objectless images
df2=df.query('num_bbox==0')
df2=df2.sample(n=800)
df=df.query('num_bbox>0')
df=pd.concat([df,df2],ignore_index=True)

df = df.sample(frac=1).reset_index(drop=True)
print(df.shape[0])

In [ ]:
df.head(2)

Storing Images

In [ ]:
# Populating the images folder
for i in tqdm(range(df.shape[0])):
    src_path=df.iloc[i]['original_path']
    dst_path=df.iloc[i]['image_path']
    
    if not os.path.exists(dst_path):
        shutil.copy(src=src_path,dst=dst_path)

In [ ]:
# Sanity check random image
ind=random.randint(0,df.shape[0])
imid=df.iloc[ind]['image_id']
plt.figure(figsize=(10, 10))
img_sample = cv2.imread("../images/"+f'{imid}'+'.jpg')
img_sample = cv2.cvtColor(img_sample, cv2.COLOR_BGR2RGB)
print("Image id is {}".format(imid))
plt.imshow(img_sample)
plt.axis("off")

Creating Labels

In [ ]:
def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    if len(bboxes)==0:
        return []
    bboxes=coco2yolo(bboxes)
    return bboxes
def coco2yolo(bboxes):
    ##yolo format [xmid, ymid, w, h]
    bboxes = np.array(bboxes).astype(float)
    # Normalize xmin, w
    bboxes[:, [0, 2]]= bboxes[:, [0, 2]]/ IMG_WIDTH
    # Normalize ymin, h
    bboxes[:, [1, 3]]= bboxes[:, [1, 3]]/ IMG_HEIGHT

    # Converstion (xmin, ymin) => (xmid, ymid)
    bboxes[:, [0, 1]] = bboxes[:, [0, 1]] + bboxes[:, [2, 3]]/2
    # Clip values (between 0 and 1)
    bboxes = np.clip(bboxes, a_min=0, a_max=1)
    return bboxes


df['bboxes_YOLO'] = df.annotations.progress_apply(get_bbox)
df.head(5)

In [ ]:
df.iloc[0]['bboxes_YOLO']

Writing down the labels

In [ ]:
# Populate the ../labels folder
yolo_bboxes=[]
for i in tqdm(range(df.shape[0])):
    row_data=df.iloc[i,:]
    yolo_bbox=row_data['bboxes_YOLO']
    
    with open(row_data['label_path'],'w') as file:
        yolo_bboxes.append(yolo_bbox)
        
        num_box=row_data['num_bbox']
        for j in range(num_box):
            annot = ["0"] + \
                    yolo_bbox[j].astype(str).tolist() + \
                    ([""] if j+1 == num_box else ["\n"])
            annot = " ".join(annot).strip()
            file.write(annot)
            

In [ ]:
# Sanity check of labels folder 
ind=random.randint(0,df.shape[0])
imid=df.iloc[ind]['image_id']
f = open("../labels/"+f'{imid}'+'.txt', 'r')
print("Image ID: {}".format(imid))
print("Number of boxes are: {}".format(df.iloc[ind]['num_bbox']))
print(f.read())

In [ ]:
## see some images with labels

# ind=random.randint(0,df.shape[0]-10)
image_list=list(df["image_id"])# videoid-sequence.jpg

ids=[]
for i in range(6):
    ids.append(image_list[random.randint(0,len(image_list))])
# video_id=[im.split("-")[0] for im in image_list]
# seq_id=[im.split("-")[1] for im in image_list]
# print(seq_id)

# Plot
paths=[]
boxnums=[]
fig, axs = plt.subplots(2, 3, figsize=(23, 10))
axs = axs.flatten()
fig.suptitle(f"Images with yolo bounding boxes", fontsize = 20)
for k in range(6):
    im=cv2.imread(f"/kaggle/images/{ids[k]}.jpg")
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    dh, dw, _ = im.shape
    path="/kaggle/labels/"+f"{ids[k]}"+".txt"
    txt=open(path,"r").read().split(" ")[1:]
    no_boxes = int(len(txt)/4)
    
    paths.append(path)
    boxnums.append(no_boxes)
#     print("###########################")
#     print(path)
#     print(txt)
#     print(no_boxes)
    
    # Draw boxes
    i = 0
    while i+4 <= 4*no_boxes:
        box = txt[i:i+4]
        i = i+4
#         print(box)
        x, y, w, h = box
        x, y, w, h = float(x), float(y), float(w), float(h)
        
        l = int((x - w / 2) * dw)
        r = int((x + w / 2) * dw)
        t = int((y - h / 2) * dh)
        b = int((y + h / 2) * dh)

        if l < 0: l = 0
        if r > dw - 1: r = dw - 1
        if t < 0: t = 0
        if b > dh - 1: b = dh - 1

        cv2.rectangle(im, (l, t), (r, b), (255,0,0), 3)
        
    axs[k].set_title(f"Sample {k}", fontsize = 14)
    axs[k].imshow(im)
    axs[k].set_axis_off()
    
plt.tight_layout()
plt.show()

print(paths)
print(boxnums)

In [ ]:
# Use Video 0 and 2 for training and 1 for validation
train_data = df[df["video_id"].isin([0, 2])]
test_data = df[df["video_id"].isin([1])]

# Get path to images & labels
train_images = list(df["image_path"])
train_labels = list(df["label_path"])

test_images = list(df["image_path"])
test_labels = list(df["label_path"])

print("Train Length:"+str(len(train_data)), "\n" +"Test Length:"+ str(len(test_data)))

In [ ]:
# shutil.rmtree("/kaggle/working")

In [ ]:
print("/kaggle/working BEFORE:"+ str(os.listdir("/kaggle/working")))

# Create train and test path data
with open("/kaggle/working/train_images.txt", "w") as file:
    for path in train_images:
        file.write(path + "\n")
        
with open("/kaggle/working/test_images.txt", "w") as file:
    for path in test_images:
        file.write(path + "\n")


# Create configuration
data = dict(path= '/kaggle/working',
          train= '/kaggle/working/train_images.txt',
          val='/kaggle/working/test_images.txt',
          nc=1,
          names= ['cots'])

with open("/kaggle/working/gbr.yaml", "w") as file:
    yaml.dump(data, file, default_flow_style=False)

        
print("/kaggle/working AFTER:"+str(os.listdir("/kaggle/working")))
f = open("/kaggle/working/gbr.yaml", 'r')
print('\nyaml:')
print(f.read())

In [ ]:
# ---> YOLOv5 install <---
%cd /kaggle/working     
!cp -r /kaggle/input/yolov5-lib-ds /kaggle/working/yolov5     
%cd yolov5     
%pip install -qr requirements.txt   

from yolov5 import utils
display = utils.notebook_init()

In [ ]:
# --- PARAMETERS ---
SIZE = 2000
BATCH_SIZE = 4
EPOCHS = 20
MODEL = "yolov5m"
WORKERS = 1
PROJECT = "GreatBarrierReef"
RUN_NAME = f"{MODEL}_size{SIZE}_epochs{EPOCHS}_batch{BATCH_SIZE}_simple"

Hypermeters

In [ ]:
%%writefile /kaggle/working/hyp.yaml
lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.40  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.10  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 0.5  # image mosaic (probability)
mixup: 0.5 # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)

In [ ]:
# Training
!python train.py --img {SIZE}\
                --batch {BATCH_SIZE}\
                --epochs {EPOCHS}\
                --data /kaggle/working/gbr.yaml\
                --hyp /kaggle/working/hyp.yaml\
                --weights {MODEL}.pt\
                --workers {WORKERS}\
                --project {PROJECT}\
                --name {RUN_NAME}\
                --exist-ok


In [ ]:
OUTPUT_DIR = '{}/{}'.format(PROJECT, RUN_NAME)
!ls {OUTPUT_DIR}

In [ ]:
# Change our position within the dirctory back
# %cd /kaggle/working

In [ ]:
# # # --- Trained Model ---
# MODEL_PATH = "../input/reef-baseline-fold12/l6_3600_uflip_vm5_f12_up/f1/best.pt"

# # Load the model
# model = torch.hub.load("../input/yolov5-lib-ds", "custom",
#                        path=MODEL_PATH,
#                        source='local', force_reload=True)

# # BoundingBox Confidence
# model.conf = 0.01
# # Intersection Over Union
# model.iou = 0.5

In [ ]:
# import greatbarrierreef

# # Initialize the environment
# env = greatbarrierreef.make_env()
# # Iterator that loops through the submission dataset
# # !!! you can run this cell only once
# iter_test = env.iter_test()

In [ ]:
# def load_model(ckpt_path, conf=0.25, iou=0.50):
#     model = torch.hub.load('/kaggle/input/yolov5-lib-ds',
#                            'custom',
#                            path=ckpt_path,
#                            source='local',
#                            force_reload=True)  # local repo
#     model.conf = conf  # NMS confidence threshold
#     model.iou  = iou  # NMS IoU threshold
#     model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
#     model.multi_label = False  # NMS multiple labels per box
#     model.max_det = 1000  # maximum number of detections per image
#     return model

In [ ]:
# ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
# # CKPT_DIR  = '/kaggle/input/greatbarrierreef-yolov5-train-ds'
# CKPT_PATH = '/kaggle/input/reef-baseline-fold12/l6_3600_uflip_vm5_f12_up/f1/best.pt' # by @steamedsheep
# IMG_SIZE  = 9000
# CONF      = 0.25
# IOU       = 0.40
# AUGMENT   = True

In [ ]:
# def predict(model, img, size=768, augment=False):
#     height, width = img.shape[:2]
#     results = model(img, size=size, augment=augment)  # custom inference size
#     preds   = results.pandas().xyxy[0]
#     bboxes  = preds[['xmin','ymin','xmax','ymax']].values
#     if len(bboxes):
#         bboxes  = voc2coco(bboxes,height,width).astype(int)
#         confs   = preds.confidence.values
#         return bboxes, confs
#     else:
#         return [],[]
    
# def format_prediction(bboxes, confs):
#     annot = ''
#     if len(bboxes)>0:
#         for idx in range(len(bboxes)):
#             xmin, ymin, w, h = bboxes[idx]
#             conf             = confs[idx]
#             annot += f'{conf} {xmin} {ymin} {w} {h}'
#             annot +=' '
#         annot = annot.strip(' ')
#     return annot

# def show_img(img, bboxes, bbox_format='yolo'):
#     names  = ['starfish']*len(bboxes)
#     labels = [0]*len(bboxes)
#     img    = draw_bboxes(img = img,
#                            bboxes = bboxes, 
#                            classes = names,
#                            class_ids = labels,
#                            class_name = True, 
#                            colors = colors, 
#                            bbox_format = bbox_format,
#                            line_thickness = 2)
#     return Image.fromarray(img).resize((800, 400))

In [ ]:
# model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
# for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
#     bboxes, confs  = predict(model, img, size=IMG_SIZE, augment=AUGMENT)
#     annot          = format_prediction(bboxes, confs)
#     pred_df['annotations'] = annot
#     env.predict(pred_df)
#     if idx<3:
#         display(show_img(img, bboxes, bbox_format='coco'))